In [1]:
import pandas as pd
import re

# File paths
telefono_csv_filepath = '/home/luisvinatea/Data/Gdrive/BERAQUA/docs/datasets/csv/telefono.csv'
email_csv_filepath = '/home/luisvinatea/Data/Gdrive/BERAQUA/docs/datasets/csv/email.csv'
location_csv_filepath = '/home/luisvinatea/Data/Gdrive/BERAQUA/docs/datasets/csv/direccion_with_links.csv'
vcf_filepath = '/home/luisvinatea/Data/Gdrive/BERAQUA/docs/datasets/vcf/contact_cards.vcf'

# Function to clean and format phone numbers
def format_phone_number(phone_number):
    # Remove all non-numeric characters except for the leading Ecuador country code
    phone_number = re.sub(r'\D', '', phone_number)
    
    # Ensure the phone number starts with Ecuador country code (+593)
    if not phone_number.startswith('593'):
        phone_number = '593' + phone_number.lstrip('0')  # Add country code and remove any leading zero

    return phone_number

# Read the CSVs into pandas DataFrames
df_telefono = pd.read_csv(telefono_csv_filepath)
df_email = pd.read_csv(email_csv_filepath)
df_location = pd.read_csv(location_csv_filepath)

# Function to read existing VCF data
def read_existing_vcf(vcf_filepath):
    with open(vcf_filepath, 'r') as vcf_file:
        vcf_data = vcf_file.read().split('END:VCARD\n')
    return vcf_data

# Read existing VCF data
existing_vcf_data = read_existing_vcf(vcf_filepath)

# Extract the importers and check if they exist in the current VCF file
existing_importers = [entry.split("\n")[1].split(":")[1] for entry in existing_vcf_data if "FN:" in entry]

# Initialize the new VCF data
vcf_data = []

# Process the phone numbers (from telefono.csv)
for index, row in df_telefono.iterrows():
    importer = row['Importer']
    phone_numbers = []
    
    # Extract phone numbers (telephone_1, telephone_2, etc.)
    for i in range(1, len(row)):
        phone_number = row[i]
        
        # Skip if the phone number is 'nan' or empty
        if pd.isna(phone_number) or phone_number == 'nan' or not phone_number.strip():
            continue
        
        # Clean and format the phone number
        formatted_phone_number = format_phone_number(phone_number)
        phone_numbers.append(formatted_phone_number)
    
    # Process email from email.csv for the same importer
    email = df_email.loc[df_email['Importer'] == importer, 'email_1'].values
    email = email[0] if len(email) > 0 else None

    # Process location (Google Maps link) from direccion_with_links.csv for the same importer
    location_link = df_location.loc[df_location['Importer'] == importer, 'google_maps_link'].values
    location_link = location_link[0] if len(location_link) > 0 else None

    # Check if this importer already exists in the VCF
    vcf_entry = None
    for entry in existing_vcf_data:
        if f"FN:{importer}" in entry:
            vcf_entry = entry
            break

    # If the importer doesn't exist in the VCF, create a new entry
    if not vcf_entry:
        vcf_entry = f"BEGIN:VCARD\nVERSION:3.0\nFN:{importer}\n"
    
    # Add phone numbers to the VCF entry
    for phone_number in phone_numbers:
        vcf_entry += f"TEL:{phone_number}\n"
    
    # Add email to the VCF entry
    if email:
        vcf_entry += f"EMAIL:{email}\n"
    
    # Add Google Maps link to the VCF entry (if available)
    if location_link:
        vcf_entry += f"URL:{location_link}\n"
    
    vcf_entry += "END:VCARD\n"

    # Append the VCF entry to the final data
    vcf_data.append(vcf_entry)

    # Optionally print to track progress
    print(f"Processed importer: {importer}")

# Write the updated VCF file with phone numbers, emails, and Google Maps links
with open(vcf_filepath, 'w') as vcf_file:
    vcf_file.writelines(vcf_data)

# Print confirmation
print(f"VCF file updated with phones, emails, and Google Maps links. Saved to: {vcf_filepath}")


Processed importer: acuarios_del_mar_s.a._acdelmar
Processed importer: allamericancorp_s.a.
Processed importer: aquagroup_cia._ltda.
Processed importer: arboleda_faini_juan_xavier
Processed importer: automotores_de_francia_compañia_anonima_autofranci
Processed importer: automotores_y_anexos_s.a._ayasa
Processed importer: camaronera_agromarina
Processed importer: camarosursa_sa
Processed importer: campoverde_palacios_marco_vinicio
Processed importer: comercial_kywi_s.a.
Processed importer: consorcio_generacion_ecuador
Processed importer: corpiecam_cia_ltda
Processed importer: crustaceos_&_peces_de_sudamerica_s.a._crupesa
Processed importer: crustaceos_&_peces_de_sudamerica_s_a_crupesa
Processed importer: dolca_s.a.
Processed importer: dreu_sa
Processed importer: ecuacobre_fv.s.a
Processed importer: ecuayamata_s.a
Processed importer: ecuayamata_s.a.
Processed importer: edesa_sa
Processed importer: electro_ecuatoriana_s.a_c.i
Processed importer: empresa_publica_de_hidrocarburos_del_ecuado

/tmp/ipykernel_199996/3836965756.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  phone_number = row[i]
